In [1]:
import sys
sys.path.append('../../pyutils')

import numpy as np
from scipy.linalg import solve_triangular

import metrics

# Definition

Let $A \in \mathbb{R}^{n*n}$  
$A$ is inversible iff it exists a matrix denoted $A^{-1} \in \mathbb{R}^{n*n}$, called the inverse of $A$, such that:
$$AA^{-1} = A^{-1}A = I_n$$

A matrix that is not inversible is called a singular matrix.

In [5]:
A = np.random.randn(4, 4)
Ai = np.linalg.inv(A)

print(A)
print(Ai)

print(metrics.tdist(A @ Ai, np.eye(4)))
print(metrics.tdist(Ai @ A, np.eye(4)))

[[ 0.60630553  0.93127023 -0.40118604 -2.89004315]
 [ 1.06221503  0.01206272  0.97233158 -1.07325908]
 [-0.43327464 -0.72019161 -1.33276614 -0.66945338]
 [ 0.901832   -1.10465848 -1.75269344 -1.06234581]]
[[ 0.03541159 -0.14674323 -1.08587007  0.73619328]
 [ 0.5017093  -0.88685637 -1.0590419   0.19847051]
 [-0.20827848  0.73736375  0.49535284 -0.49048453]
 [-0.14800623 -0.41891908 -0.63782849  0.28648821]]
5.551115123125783e-16
8.635508522760763e-16


# Inverse and system of equations

We can define a sytem of $n$ equations with $n$ unknowns as the matrix-vector product:
$$Ax = b$$
with $A \in \mathbb{R}^{n*n}$ the lhs terms, $b \in \mathbb{R}^n$ the rhs terms, and $x \in \mathbb{R}^n$ the unknowns.

$A$ is inversible iff there is exactly one solution to $Ax = b$ for every possible $b$.
$$x = A^{-1}b$$

In [8]:
A = np.random.randn(4, 4)
b = np.random.randn(4)
x = np.linalg.inv(A) @ b

print(x)
print(metrics.tdist(A @ x, b))

[ 1.4675494  -3.31991027  1.49758185 -3.81867478]
8.08254562088053e-16


# Inverse and linear combinations

The inversibility of $A$ can be stated as any of these properties:
- The columns of $A$ are linearly independant
- $\text{Col}(A) = R^n$
- $\text{rank}(A) = n$
- $\text{Null}(A) = \vec{0}$

# The moore-Penrose Pseudoinverse

The pseudoinverse is a generalization of inverse that can be applied to any matrix.

Let $A \in \mathbb{R}^{n*m}$
The pseudoinverse of $A$ is denoted $A^+ \in \mathbb{R}^{m,n}$.  
It is defined as:
$$A^+ = \lim_{\alpha \to 0} (A^TA + \alpha I)^{-1} A^T$$

If $A$ is inversible, we get $A^+ = A^{-1}$

In [19]:
A = np.random.randn(4, 5)
Ai = np.linalg.pinv(A)
print(metrics.tdist(A @ Ai, np.eye(4)))
print(metrics.tdist(Ai @ A, np.eye(5)))

1.1315819425944535e-15
0.9999999999999999


In [20]:
A = np.random.randn(4, 4)
Ai = np.linalg.inv(A)
Ap = np.linalg.pinv(A)
print(metrics.tdist(Ai, Ap))

6.434452302420132e-15


## Computations

The pseudoinverse is usually computed with the SVD decomposition
$$A = UDV^T$$
$$A^+ = VD^+ U^T$$

with $D^+$ the transpose of the matrix containing the reciprocal of the non-zero values of $D$.

In [47]:
def my_pinv(A, tol=1e-8):
    U, d, VT = np.linalg.svd(A, full_matrices=False)
    Dp = np.diag([0 if x < tol else 1/x for x in d])
    return VT.T @ Dp @ U.T

A = np.random.randn(5, 4)
Ap1 = my_pinv(A)
Ap2 = np.linalg.pinv(A)

print(metrics.tdist(Ap1, Ap2))
print(metrics.tdist(A @ Ap1, np.eye(A.shape[0])))
print(metrics.tdist(Ap1 @ A, np.eye(A.shape[1])))

1.8041124150158794e-16
1.0
1.4816321196292979e-15


## System of equations

Let the system $Ax = b$ with $A \in \mathbb{R}^{n*m}$, $x \in \mathbb{R}^m$, and $b \in \mathbb{R}^n$. 
Using the pseudo-inverse, we get the solution:
$$\hat{x} = A^+b$$

If $A$ has linearnly independant columns, they are infinitely solutions, and we get the least-norm solution:
$$\hat{x} = \min_{x} ||x||_2$$
$$\text{s.t. } Ax=b$$

If $A$ has linearly independant rows, they may be no solutions, and we get the least squares solution:
$$\hat{x} = \min_x ||Ax - b||_2$$

In [52]:
#linearly dependent columns: infinitely many solutions
A = np.random.randn(4, 5)
b = np.random.randn(4)
x = np.linalg.pinv(A) @ b

print(metrics.tdist(A @ x, b))
print(np.linalg.norm(x))

1.9373312250675786e-15
2.270219206357372


In [58]:
#linearly dependent rows: no solution
A = np.random.randn(5, 4)
b = np.random.randn(5)
x = np.linalg.pinv(A) @ b

print(metrics.tdist(A @ x, b))

0.47274831094063247


# Determinant

Let $A \in \mathbb{R}^{n*n}$.  
The determinant of $A$, noted $\text{det}(A)$ is a mesure of volume.

$$\text{det}(A) = \prod_{i=1}^n \lambda_i$$ 
with $\lambda_i$ eigenvalue of $A$.

A singular matrix has a determinant of $0$.

In [60]:
A = np.random.randn(4, 4)
det = np.linalg.det(A)

w, V = np.linalg.eig(A)
wp = np.prod(w)

print(det)
print(wp)

-3.1845197649245587
(-3.1845197649245556+2.9464398150217424e-16j)
